In [1]:
from google.colab import drive

In [2]:
drive.mount('/gdrive', force_remount= True)

Mounted at /gdrive


In [3]:
import glob, pickle
import numpy as np
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential

from keras.layers import Dense, Dropout, LSTM, CuDNNLSTM

from keras.utils import np_utils

In [6]:
#midi_songs에 있는 Note 정보만 뽑아서 저장할 리스트
url = '/gdrive/My Drive/캡스톤 디자인/'
file_name = url +'midi_major_song/장재영/*'
notes = []

for i, file in enumerate(glob.glob(file_name)):
  midi =  converter.parse(file)
  print('\r', 'Parsing file', i, " ",file, end='')

  #Midi file을 notes로 나누어 다루기 위한 변수
  # notes_to_parse = None
  right_notes = None
  # #try / except : try 수행 중 에러 발생 시 except 수행

  # try :# file has instrument parts
  #   s2 = instrument.partitionByInstrument(midi)
  #   notes_to_parse = s2.parts[0].recurse()
  # except: # file has notes in a flat structure
  #   notes_to_parse = midi.flat.notes

  parts = midi.getElementsByClass(stream.Part)
  right_notes = parts[1].flat.notes

  for e in right_notes:

    # Note 인 경우 높이(Pitch), Octave 로 저장
    if isinstance(e, note.Note):
      notes.append(str(e.pitch))

     # Chord 인 경우 각 Note의 음높이(Pitch)를 '.'으로 나누어 저장
    elif isinstance(e, chord.Chord):
      # ':'.join([0, 1, 2]) : [0, 1, 2] -> [0:1:2]
      # str(n) for n in e.normalOrder 
      #     => e.normalOrder 라는 배열 내의 모든 원소 n에 대해 str(n) 해준 새 배열을 만든다.
      #        ex) str(i) for i in [1, 2, 3] => ['1', '2', '3']
      notes.append('.'.join(str(n) for n in e.normalOrder))

 Parsing file 49   /gdrive/My Drive/캡스톤 디자인/midi_major_song/장재영/Beethoven_-_Minuet_in_G_Major_WoO_10_No._2.mid

In [7]:
# MIDI 파일 정보를 다루기 쉽게 바꿔준다
# n_vocab : 모델 출력의 가짓수를 정하기 위해 Note의 총 가짓수를 센다(Note의 종류)
# set() : 중복되는 원소는 한번만 쓴다 / ex) set("Hello") => {'e', 'H', 'l', 'o'}
n_vocab = (len(set(notes)))

print('Classes of notes : ', n_vocab, '\n')
print('notes : ', notes[:500])
print('length of notes : ', len(notes), '\n')

# pitchnames : notes 배열의 모든 가능한 Note / Chord 를 정렬해놓은 배열  

pitchnames = sorted(set(item for item in notes))

print('pitchnames : ', pitchnames)
print('length of pitchnames : ', len(pitchnames), '\n')

# create a dictionary to map pitches to integers
# 음높이(Pitch)를 정수에 매핑하는 dictionary 자료형 생성
# ex) dict = {'key': value} => dict['key'] = value
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

print('note_to_int : ', note_to_int)

Classes of notes :  281 

notes :  ['F2', 'A2', 'C3', 'F3', 'A3', 'C4', 'F4', 'A4', 'C5', 'B-4', 'A4', 'B-4', 'A4', 'A4', 'F4', 'F4', 'F4', 'F3', 'F3', 'F3', 'F2', 'F2', 'F2', 'G2', 'F2', 'G2', 'F2', '0.5', '0.5', '0.5', '10.2.5', '10.2.5', '10.2.5', '10.2.5', '10.2.5', '10.2.5', 'F3', 'F3', 'F3', 'F4', 'D4', 'E4', 'F4', 'G4', 'E4', 'F4', 'G4', 'A4', 'F4', 'G4', 'A4', 'B4', 'G4', 'B4', 'C5', 'F4', 'F#4', 'G4', 'F3', '0.3.7', '0.3.7', '0.3.7', 'B-3', 'B-3', 'B-3', 'B-3', 'B-3', 'B-3', '8.0', '8.0', '8.0', '8.0', '8.0', '8.0', 'G3', 'G3', 'G3', 'G3', '2.7', '7.0', 'G3', 'G2', '2.7', '2.7', '2.7', '2.5', '2.5', '0.3', '11.2', '11.2', '8.0', '7.11', '7.11', '7.9.0', '2.7', '7.9.0', '7.11.2', '0.4', '0.2.5', '0.4.7', 'F3', 'G3', '7.11.2', 'C4', 'C4', 'C4', 'C4', 'C3', 'C3', 'C3', 'C2', 'C2', 'C2', 'D2', 'C2', 'D2', 'C2', 'F2', 'A2', 'C3', 'F3', 'A3', 'C4', 'F4', 'A4', 'C5', 'B-4', 'A4', 'B-4', 'A4', 'A4', 'F4', 'F4', 'F4', 'F3', 'F3', 'F3', 'F2', 'F2', 'F2', 'G2', 'F2', 'G2', 'F2', '0.5', '

In [8]:
# LSTM 모델을 위한 Training Dataset 생성
# 곡을 배끼는 것이 아닌 작곡이기 때문에 Validation이 필요없다

seq_len = 100 # 시퀀스 길이

# Pitch를 정수로 바꾸어 LSTM 모델의 입출력으로 만들어준다

net_in = []
net_out = []

# LSTM 모델의 입출력을 만들기 위해 ( 전체 길이 - 시퀀스 길이(=100) ) 만큼 반복
# ex) 입력 : 출력 짝지어주기
#     [0 ~ 99] : [100] / [1 ~ 100] : [101] / ... / [전체 길이-100 ~ 전체 길이-1] : [전체 길이]
for i in range(0, len(notes) - seq_len):
  # LSTM 모델 입력과 출력을 만들어준다
  seq_in = notes[i:i + seq_len] # ex) [0:100] => [0 ~ 99]
  seq_out = notes[i + seq_len]  # ex) [100]
  # LSTM은 문자열이 아닌 숫자를 입출력으로 하므로 문자열을 정수로 바꿔야 한다
  net_in.append([note_to_int[char] for char in seq_in]) # 배열 안의 모든 원소에 대해 실행
  net_out.append(note_to_int[seq_out]) # 출력값 하나에 대해 실행
print(np.shape(net_in))
print(np.shape(net_out))

(20072, 100)
(20072,)


In [9]:
# LSTM 모델 입출력에 맞게 Dataset 전처리

# 시퀀스 길이(100) 만큼을 빼고 반복했으므로 100개 적은 패턴이 생긴다
n_patterns = len(net_in)
print('n_patterns : ', n_patterns)

# reshape the input into a format compatible with LSTM layers
# LSTM 입력에 맞는 모양으로 바꿔준다 : (샘플 수, 시퀀스 길이, 자료의 차원)
net_in = np.reshape(net_in, (n_patterns, seq_len, 1))
print('shape of net_in : ', net_in.shape)

# 데이터 범위 정규화 : 0 ~ (n_vocab - 1) => 0 ~ 1
net_in = net_in / float(n_vocab)

# 분류이므로 출력을 One-hot Vector로 만들어주어야 한다.
net_out = np_utils.to_categorical(net_out)
print('shape of net_out : ', net_out.shape)


n_patterns :  20072
shape of net_in :  (20072, 100, 1)
shape of net_out :  (20072, 281)


In [10]:
# 모델 구성

# 데이터의 Feature(특징) 수 or Dimension(차원)
data_dim = net_in.shape[2]

# GPU 환경 : CuDNNLSTM() / CPU 환경 : LSTM()
model = Sequential(name="Melody_LSTM")

# return_sequences : True : Many to Many / False : Many to One
# seq_len : 입력으로 넣을 시계열 데이터의 길이 / data_dim : 각 데이터의 차원

model.add(CuDNNLSTM(512, input_shape=(seq_len, data_dim), return_sequences=True))

# GPU / CUDA / CuDNN 이 없는 환경에선 CuDNNLSTM만 LSTM으로 바꾸어 쓰면 됩니다.
# model.add(LSTM(512, input_shape=(seq_len, data_dim), return_sequences=True))
model.add(Dropout(rate=0.3))

model.add(CuDNNLSTM(512, return_sequences=True))
model.add(Dropout(rate=0.3))

model.add(CuDNNLSTM(512))

model.add(Dense(256))
model.add(Dropout(rate=0.3))

model.add(Dense(n_vocab, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()


Model: "Melody_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cu_dnnlstm (CuDNNLSTM)      (None, 100, 512)          1054720   
                                                                 
 dropout (Dropout)           (None, 100, 512)          0         
                                                                 
 cu_dnnlstm_1 (CuDNNLSTM)    (None, 100, 512)          2101248   
                                                                 
 dropout_1 (Dropout)         (None, 100, 512)          0         
                                                                 
 cu_dnnlstm_2 (CuDNNLSTM)    (None, 512)               2101248   
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout_2 (Dropout)         (None, 256)               

In [ ]:
# 모델 학습
# 음악 작곡이기 때문에 Validation Set이 없다
# LSTM : 13시간 / CuDNN : 3시간 반
# loss 는 0.7 ~ 0.8 만 되어도 충분히 작곡 능력을 갖는다.
model.fit(net_in, net_out, epochs=70, batch_size=64)

Epoch 1/70
314/314 [==============================] - 62s 191ms/step - loss: 4.4604
Epoch 2/70
314/314 [==============================] - 60s 190ms/step - loss: 4.3748
Epoch 3/70
314/314 [==============================] - 60s 191ms/step - loss: 4.3583
Epoch 4/70
314/314 [==============================] - 60s 190ms/step - loss: 4.3270
Epoch 5/70
314/314 [==============================] - 60s 191ms/step - loss: 4.2380
Epoch 6/70
314/314 [==============================] - 60s 191ms/step - loss: 4.1453
Epoch 7/70
314/314 [==============================] - 61s 193ms/step - loss: 4.0782
Epoch 8/70
314/314 [==============================] - 61s 193ms/step - loss: 4.0567
Epoch 9/70
314/314 [==============================] - 61s 193ms/step - loss: 4.0525
Epoch 10/70
314/314 [==============================] - 61s 193ms/step - loss: 4.0420
Epoch 11/70
314/314 [==============================] - 61s 193ms/step - loss: 4.0270
Epoch 12/70
314/314 [==============================] - 61s 193ms/step - lo

In [ ]:
# 작곡을 위해 LSTM 모델 입력을 다시 만든다

# 위와 동일하므로 주석 생략

net_in = []
output = []

for i in range(0, len(notes) - seq_len, 1):
  seq_in = notes[i:i + seq_len]
  seq_out = notes[i + seq_len]

  net_in.append([note_to_int[char] for char in seq_in])
  output.append(note_to_int[seq_out])
  
n_patterns = len(net_in)

In [ ]:
# LSTM 모델이 작곡을 시작하기 위해 시작점으로써 랜덤한 시퀀스를 골라야 한다

# pattern : Dataset의 입력 전체 시퀀스 중 랜덤하게 고른 시퀀스
start = np.random.randint(0, len(net_in)-1)
pattern = net_in[start]
print('Random Sequence : ', pattern)

# int_to_note: 정수를 다시 Note로 바꾸기 위한 dictionary 자료형
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
print('int_to_note : ', int_to_note)

Random Sequence :  [270, 254, 214, 271, 254, 231, 214, 106, 193, 123, 202, 277, 265, 247, 241, 242, 237, 220, 214, 157, 214, 220, 277, 214, 259, 277, 241, 242, 237, 220, 214, 277, 265, 247, 241, 242, 237, 226, 214, 59, 277, 237, 214, 265, 220, 247, 139, 220, 219, 277, 265, 247, 241, 242, 237, 220, 214, 157, 214, 220, 277, 214, 259, 277, 241, 242, 237, 220, 214, 277, 265, 247, 241, 242, 237, 226, 214, 59, 277, 237, 214, 265, 220, 247, 139, 220, 219, 220, 214, 277, 265, 277, 253, 236, 265, 214, 277, 265, 277, 265]
int_to_note :  {0: '0', 1: '0.1.5', 2: '0.1.5.8', 3: '0.2', 4: '0.2.5', 5: '0.2.5.6', 6: '0.2.6', 7: '0.2.7', 8: '0.3', 9: '0.3.4.7', 10: '0.3.5.8', 11: '0.3.6', 12: '0.3.6.8', 13: '0.3.6.9', 14: '0.3.7', 15: '0.4', 16: '0.4.5', 17: '0.4.7', 18: '0.5', 19: '0.6', 20: '1', 21: '1.2', 22: '1.2.6.9', 23: '1.3', 24: '1.3.7', 25: '1.4', 26: '1.4.5.8', 27: '1.4.6', 28: '1.4.6.9', 29: '1.4.7', 30: '1.4.7.10', 31: '1.4.7.9', 32: '1.4.8', 33: '1.5', 34: '1.5.8', 35: '1.6', 36: '1.7', 37

In [ ]:
# LSTM 모델이 만든 출력값을 저장하기 위한 빈 리스트

pred_out = []

# generate 500 notes
for i in range(0, 500):
  # 랜덤하게 고른 시퀀스를 LSTM 모델 입력에 맞게 바꿔준다
  pred_in = np.reshape(pattern, (1, len(pattern), 1))

  # 입력 범위 정규화 / 0 ~ (n_vocab -1) => 0 ~ 1
  pred_in = pred_in / float(n_vocab)

  # LSTM 모델 사용
  prediction = model.predict(pred_in, verbose=0)

  # 출력 중 값이 가장 큰 Index 선택
  index = np.argmax(prediction)

  # 정수 값을 Note 값으로 변경
  result = int_to_note[index]
  print('\r', 'Predicted ', i, " ",result, end='')
  
  # LSTM이 만든 Note를 하나씩 리스트에 담는다
  pred_out.append(result)
  
  # 다음 입력을 위해 입력에 새 값 추가 후 가장 과거 값 제거
  # ex) [0:99] -> [1:100] -> ... -> [n : n + 99]
  pattern.append(index)
  pattern = pattern[1:len(pattern)]

 Predicted  499   E3

In [ ]:
print('length of pred_out : ', len(pred_out))
print('pred_out : ', pred_out)

length of pred_out :  500
pred_out :  ['E-3', 'D3', 'E-3', 'F3', 'B-2', 'D4', 'C4', '11.2', 'C4', 'B-3', 'A3', 'G3', 'D4', 'A3', 'G3', 'F#3', 'E3', 'D3', 'E-3', 'D3', 'C3', 'B-2', 'C3', 'D3', 'G3', 'G2', 'B-3', 'A3', 'G3', 'F3', 'G3', 'E3', 'C3', 'F3', 'A3', 'G3', 'F3', 'G3', 'F3', 'E-3', 'D3', 'E-3', 'F3', 'B-2', 'D4', 'C4', '11.2', 'C4', 'B-3', 'A3', 'G3', 'D4', 'A3', 'G3', 'F#3', 'E3', 'D3', 'E-3', 'D3', 'C3', 'B-2', 'C3', 'D3', 'G3', 'G2', 'E4', 'G4', 'E4', 'B3', 'G4', 'C#4', 'A3', 'E4', 'C#4', 'D4', 'C#4', 'B3', 'G#3', 'E4', 'B3', 'G3', 'E4', 'B3', 'E4', 'B3', 'G#3', 'F#4', 'E-4', 'B-3', 'G#4', 'E4', 'C#4', 'B3', 'C#4', 'E-4', 'E4', 'C#4', 'F#4', 'E-4', 'C#4', 'G4', 'E4', 'B3', 'G4', 'E4', 'C#4', 'G4', 'E4', 'B3', 'G4', 'E-4', 'C#4', 'G4', 'E4', 'B3', 'F#4', 'E-4', 'B-3', 'F#4', 'E-4', 'B3', 'F#4', 'E-4', 'B3', 'E4', 'E-4', 'B3', 'G4', 'E4', 'B3', 'G4', 'E4', 'C#4', 'G4', 'E4', 'B3', 'F#4', 'E-4', 'B-3', 'F#4', 'B3', 'F#4', 'E-4', 'B3', 'F#4', 'E-4', 'C#4', 'G4', 'E4', 'C#4', 'G4'

In [ ]:
# LSTM 모델이 예측한 값들로부터 MIDI 파일을 만들어준다

offset = 0 # 음(Note/Chord)을 언제 들려줄지 정하는 timing offset (박자 정보를 대신 함

# MIDI 파일 생성을 위한 빈 리스트
output_notes = []

# create note and chord objects based on the values generated by the model
# LSTM 모델 예측 값을 하나씩 처리
for pattern in pred_out:

    # pattern이 Chord 일 때
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.') # ['8.1'].split('.') => ['8', '1']
        notes = [] # Note 정보를 저장할 빈 리스트

        # notes_in_chord의 텍스트를 Note 정보로 바꿔준다
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note)) # Text => 정수 => Note
            new_note.storedInstrument = instrument.Piano() # 악기 정보 설정
            notes.append(new_note) # notes 리스트에 더해준다

        # Note => Chord
        new_chord = chord.Chord(notes)
        new_chord.offset = offset # 시간 정보 설정
        output_notes.append(new_chord)
    # pattern이 Note 일 때
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    # 반복마다 offset을 증가시킨다 (고정 박자)
    offset += 0.5

# Note/Chord => Stream => MIDI File
storage_path = url 
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp= storage_path)

'/gdrive/My Drive/melody.mid'

In [ ]:
# 모델은 h5 파일 형태로 저장됩니다
# 경로에 주의합시다

model.save(storage_path+'Melody_LSTM.h5')
# 모델을 불러오기 위해 지워줍니다
del model
# 저장되어 있는 모델을 불러오기 위한 load_model 함수
from keras.models import load_model
# 'model' 에 해당 모델을 불러옵니다
model = load_model(storage_path+'Melody_LSTM.h5')